In [1]:
#First we load the basic necessary packages
import pandas as pd
import numpy as np
import scipy.stats as ss
import copy
import matplotlib.pyplot as plt
import pandas_profiling
import sklearn.preprocessing as preprocessing
import seaborn as sns
import sklearn.model_selection as cross_validation
import sklearn.linear_model as linear_model
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix

import scipy
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

from pylab import rcParams
import seaborn as sb

import sklearn
from sklearn.cluster import AgglomerativeClustering
import sklearn.metrics as sm

In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
np.set_printoptions(precision=4, suppress=True)
plt.figure(figsize=(10,3))
plt.style.use('seaborn-whitegrid')

<Figure size 720x216 with 0 Axes>

In [3]:
elog = pd.read_csv("BPI Challenge 2017 - e.csv", sep=",")

In [4]:
group_act = elog.groupby('Activity')['Resource'].count()

In [5]:
print(group_act)

Activity
A_Accepted                    31509
A_Cancelled                   10431
A_Complete                    31362
A_Concept                     31509
A_Create Application          31509
A_Denied                       3753
A_Incomplete                  23055
A_Pending                     17228
A_Submitted                   20423
A_Validating                  38816
O_Accepted                    17228
O_Cancelled                   20898
O_Create Offer                42995
O_Created                     42995
O_Refused                      4695
O_Returned                    23305
O_Sent (mail and online)      39707
O_Sent (online only)           2026
W_Assess potential fraud        355
W_Call after offers           31485
W_Call incomplete files       23218
W_Complete application        29918
W_Handle leads                 3727
W_Personal Loan collection        4
W_Shortened completion           76
W_Validate application        39444
Name: Resource, dtype: int64


In [6]:
group_elog = elog.groupby(['Resource','Activity']).agg({'Activity':'count'})
print(group_elog)

                                   Activity
Resource Activity                          
User_1   A_Cancelled                   7953
         A_Concept                    16997
         A_Create Application         20423
         A_Submitted                  20423
         O_Cancelled                   9982
...                                     ...
User_99  W_Assess potential fraud         2
         W_Call after offers              8
         W_Call incomplete files       1097
         W_Complete application           6
         W_Validate application        1677

[2305 rows x 1 columns]


In [7]:
group_elog.columns=['Count']

group_elog = group_elog.reset_index()

Udh hitung jumlah aktivitas yang dilakukan, bisa hitung berapa distance antara user based on activities.

In [8]:
print(group_elog)

     Resource                  Activity  Count
0      User_1               A_Cancelled   7953
1      User_1                 A_Concept  16997
2      User_1      A_Create Application  20423
3      User_1               A_Submitted  20423
4      User_1               O_Cancelled   9982
...       ...                       ...    ...
2300  User_99  W_Assess potential fraud      2
2301  User_99       W_Call after offers      8
2302  User_99   W_Call incomplete files   1097
2303  User_99    W_Complete application      6
2304  User_99    W_Validate application   1677

[2305 rows x 3 columns]


In [9]:
ggroup_elog = group_elog

In [10]:
ggroup_elog = ggroup_elog.groupby('Resource')['Count'].sum()

In [11]:
ggroup_elog.columns='Sum'
ggroup_elog = ggroup_elog.reset_index()

In [12]:
print(ggroup_elog)

     Resource  Count
0      User_1  75950
1     User_10   9824
2    User_100   7590
3    User_101   2342
4    User_102   3849
..        ...    ...
140   User_95   4143
141   User_96   1681
142   User_97   1034
143   User_98    794
144   User_99   6337

[145 rows x 2 columns]


In [13]:
result2 = pd.merge(group_elog, ggroup_elog, how='outer', on='Resource')
result = pd.merge(result2, group_act, how='outer', on='Activity')

In [14]:
print(result)

     Resource_x                    Activity  Count_x  Count_y  Resource_y
0        User_1                 A_Cancelled     7953    75950       10431
1       User_10                 A_Cancelled       79     9824       10431
2      User_102                 A_Cancelled        1     3849       10431
3      User_103                 A_Cancelled        2       73       10431
4      User_104                 A_Cancelled        5     1743       10431
...         ...                         ...      ...      ...         ...
2300    User_49      W_Shortened completion        5    10832          76
2301     User_5      W_Shortened completion        2     8636          76
2302    User_53      W_Shortened completion        2     5073          76
2303   User_138  W_Personal Loan collection        2      343           4
2304    User_50  W_Personal Loan collection        2      358           4

[2305 rows x 5 columns]


In [15]:
result['Percentage_Ind'] = result.apply(lambda row: row.Count_x/row.Count_y, axis=1)
result['Percentage_Act'] = result.apply(lambda row: row.Count_x/row.Resource_y, axis=1)

In [16]:
print(result)

     Resource_x                    Activity  Count_x  Count_y  Resource_y  \
0        User_1                 A_Cancelled     7953    75950       10431   
1       User_10                 A_Cancelled       79     9824       10431   
2      User_102                 A_Cancelled        1     3849       10431   
3      User_103                 A_Cancelled        2       73       10431   
4      User_104                 A_Cancelled        5     1743       10431   
...         ...                         ...      ...      ...         ...   
2300    User_49      W_Shortened completion        5    10832          76   
2301     User_5      W_Shortened completion        2     8636          76   
2302    User_53      W_Shortened completion        2     5073          76   
2303   User_138  W_Personal Loan collection        2      343           4   
2304    User_50  W_Personal Loan collection        2      358           4   

      Percentage_Ind  Percentage_Act  
0           0.104714        0.762439

In [17]:
result['Expert'] = np.where((result['Percentage_Ind'])>result['Percentage_Act'], 1, 0)

In [18]:
print(result)

     Resource_x                    Activity  Count_x  Count_y  Resource_y  \
0        User_1                 A_Cancelled     7953    75950       10431   
1       User_10                 A_Cancelled       79     9824       10431   
2      User_102                 A_Cancelled        1     3849       10431   
3      User_103                 A_Cancelled        2       73       10431   
4      User_104                 A_Cancelled        5     1743       10431   
...         ...                         ...      ...      ...         ...   
2300    User_49      W_Shortened completion        5    10832          76   
2301     User_5      W_Shortened completion        2     8636          76   
2302    User_53      W_Shortened completion        2     5073          76   
2303   User_138  W_Personal Loan collection        2      343           4   
2304    User_50  W_Personal Loan collection        2      358           4   

      Percentage_Ind  Percentage_Act  Expert  
0           0.104714        

In [19]:
allresult = pd.DataFrame(result)

In [20]:
allresult

,Resource_x,Activity,Count_x,Count_y,Resource_y,Percentage_Ind,Percentage_Act,Expert
0,User_1,A_Cancelled,7953,75950,10431,0.104714,0.762439,0
1,User_10,A_Cancelled,79,9824,10431,0.008042,0.007574,1
2,User_102,A_Cancelled,1,3849,10431,0.000260,0.000096,1
3,User_103,A_Cancelled,2,73,10431,0.027397,0.000192,1
4,User_104,A_Cancelled,5,1743,10431,0.002869,0.000479,1
...,...,...,...,...,...,...,...,...
2300,User_49,W_Shortened completion,5,10832,76,0.000462,0.065789,0
2301,User_5,W_Shortened completion,2,8636,76,0.000232,0.026316,0
2302,User_53,W_Shortened completion,2,5073,76,0.000394,0.026316,0
2303,User_138,W_Personal Loan collection,2,343,4,0.005831,0.500000,0


In [21]:
xresult = pd.DataFrame(result, columns=['Resource_x','Activity','Expert'])

In [22]:
xresult.rename(columns = {'Resource_x':'Resource'}, inplace = True)

In [23]:
xresult

,Resource,Activity,Expert
0,User_1,A_Cancelled,0
1,User_10,A_Cancelled,1
2,User_102,A_Cancelled,1
3,User_103,A_Cancelled,1
4,User_104,A_Cancelled,1
...,...,...,...
2300,User_49,W_Shortened completion,0
2301,User_5,W_Shortened completion,0
2302,User_53,W_Shortened completion,0
2303,User_138,W_Personal Loan collection,0


In [24]:
xresult2 = pd.merge(elog, xresult, how='outer', on=['Resource','Activity'])

In [25]:
xresult2

,Case ID,Activity,Resource,Start Timestamp,Complete Timestamp,Variant,Variant index,(case) ApplicationType,(case) LoanGoal,(case) RequestedAmount,...,EventID,EventOrigin,FirstWithdrawalAmount,MonthlyCost,NumberOfTerms,OfferID,OfferedAmount,Selected,lifecycle:transition,Expert
0,Application_652823628,A_Create Application,User_1,2016/01/01 10:51:15.304,2016/01/01 10:51:15.304,Variant 2,2,New credit,Existing loan takeover,20000.0,...,Application_652823628,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,0
1,Application_1691306052,A_Create Application,User_1,2016/01/01 11:16:11.500,2016/01/01 11:16:11.500,Variant 388,388,New credit,Home improvement,10000.0,...,Application_1691306052,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,0
2,Application_428409768,A_Create Application,User_1,2016/01/01 12:19:38.177,2016/01/01 12:19:38.177,Variant 209,209,New credit,Home improvement,15000.0,...,Application_428409768,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,0
3,Application_1746793196,A_Create Application,User_1,2016/01/01 13:34:53.911,2016/01/01 13:34:53.911,Variant 256,256,New credit,Car,5000.0,...,Application_1746793196,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,0
4,Application_828200680,A_Create Application,User_1,2016/01/01 14:00:04.360,2016/01/01 14:00:04.360,Variant 1,1,New credit,Home improvement,35000.0,...,Application_828200680,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561666,Application_1068338293,A_Create Application,User_105,2016/12/29 16:48:57.119,2016/12/29 16:48:57.119,Variant 76,76,New credit,Existing loan takeover,23000.0,...,Application_1068338293,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,1
561667,Application_753322555,A_Create Application,User_105,2016/12/29 17:33:04.775,2016/12/29 17:33:04.775,Variant 4,4,New credit,Home improvement,25000.0,...,Application_753322555,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,1
561668,Application_838340812,A_Create Application,User_105,2016/12/30 12:57:46.893,2016/12/30 12:57:46.893,Variant 4,4,New credit,"Other, see explanation",20000.0,...,Application_838340812,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,1
561669,Application_1131485787,A_Create Application,User_105,2016/12/30 16:49:37.813,2016/12/30 16:49:37.813,Variant 4039,4039,New credit,Remaining debt home,17000.0,...,Application_1131485787,Application,NaN,NaN,NaN,NaN,NaN,NaN,complete,1


In [26]:
xresult3 = pd.DataFrame(xresult2, columns=(['Case ID','Activity','Resource','Expert']))
xresult3

,Case ID,Activity,Resource,Expert
0,Application_652823628,A_Create Application,User_1,0
1,Application_1691306052,A_Create Application,User_1,0
2,Application_428409768,A_Create Application,User_1,0
3,Application_1746793196,A_Create Application,User_1,0
4,Application_828200680,A_Create Application,User_1,0
...,...,...,...,...
561666,Application_1068338293,A_Create Application,User_105,1
561667,Application_753322555,A_Create Application,User_105,1
561668,Application_838340812,A_Create Application,User_105,1
561669,Application_1131485787,A_Create Application,User_105,1


In [27]:
axa = xresult3.groupby('Case ID')['Activity'].count()

In [28]:
axe= xresult3.groupby('Case ID')['Expert'].sum()

In [29]:
axae = pd.merge(axa,axe, how='outer', on='Case ID')

In [30]:
axae

,Activity,Expert
Case ID,,
Application_1000086665,12,7
Application_1000158214,15,12
Application_1000311556,11,9
Application_1000334415,19,14
Application_1000339879,24,21
...,...,...
Application_999507989,17,14
Application_999544538,40,36
Application_999632431,12,7


In [31]:
axae['Expert_Rate'] = axae.apply(lambda row: row.Expert/row.Activity, axis=1)

In [32]:
axae

,Activity,Expert,Expert_Rate
Case ID,,,
Application_1000086665,12,7,0.583333
Application_1000158214,15,12,0.800000
Application_1000311556,11,9,0.818182
Application_1000334415,19,14,0.736842
Application_1000339879,24,21,0.875000
...,...,...,...
Application_999507989,17,14,0.823529
Application_999544538,40,36,0.900000
Application_999632431,12,7,0.583333
